In [1]:
#| default_exp datasources.telecom
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [2]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Telecomunicações
> Módulo para encapsular a extração e processamento dos dados de telecomunicações distintos do SMP - Serviço Móvel Pessoal

In [3]:
#| export
import os
from functools import cached_property

import pandas as pd
from dotenv import find_dotenv, load_dotenv

from extracao.constants import (
    AGG_LICENCIAMENTO,
    COLS_LICENCIAMENTO,
    DICT_LICENCIAMENTO,
    MONGO_TELECOM,
    PROJECTION_LICENCIAMENTO,
)

from extracao.datasources.mosaico import Mosaico

In [4]:
#| export
load_dotenv(find_dotenv())

True

In [5]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [6]:
#| export

MONGO_URI = os.environ.get("MONGO_URI")


In [7]:
#| export
class Telecom(Mosaico):
    """Extração e Processamento dos serviços de Telecomunições distintos de SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI, limit: int = 0) -> None:
        super().__init__(mongo_uri)
        self.limit = limit

    @property
    def stem(self):
        return "telecom"

    @property
    def collection(self):
        return "licenciamento"

    @property
    def query(self):
        return MONGO_TELECOM

    @property
    def projection(self):
        return PROJECTION_LICENCIAMENTO

    @property
    def columns(self):
        return COLS_LICENCIAMENTO

    @property
    def cols_mapping(self):
        return DICT_LICENCIAMENTO

    @cached_property
    def extraction(self) -> pd.DataFrame:
        pipeline = [{"$match": self.query}, {"$project": self.projection}]
        if self.limit > 0:
            pipeline.append({"$limit": self.limit})
        df = self._extract(self.collection, pipeline)
        df["Log"] = '[]'
        # Substitui strings vazias e somente com espaços por nulo
        return df.replace(r"^\s*$", pd.NA, regex=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    ) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
        """Clean the merged dataframe with the data from the MOSAICO page"""
        df = df.rename(columns=self.cols_mapping)
        df = self.split_designacao(df)
        duplicated = df.duplicated(subset=AGG_LICENCIAMENTO, keep="first")
        df_sub = df[~duplicated].reset_index(drop=True)
        discarded = df[duplicated].reset_index(drop=True)
        log = f"""[("Colunas", {AGG_LICENCIAMENTO}),  
        ("Processamento", "Registro agrupado e descartado do arquivo final")]"""
        discarded = self.register_log(discarded, log)
        df_sub["Multiplicidade"] = (
            df.groupby(AGG_LICENCIAMENTO, sort=False).size().tolist()
        )
        log = f'[("Colunas", {AGG_LICENCIAMENTO}), ("Processamento", "Agrupamento")]'
        df_sub = self.register_log(df_sub, log, df_sub.Multiplicidade > 1)
        df_sub["Status"] = "L"
        df_sub["Fonte"] = "MOSAICO"
        self.append2discarded([self.discarded, discarded])
        return df_sub.loc[:, self.columns]

In [8]:
#| export
if __name__ == "__main__":
    import time

    start = time.perf_counter()

    data = Telecom(limit=100000)

    data.update()

    print("DATA")

    display(data.df)

    print(150 * "=")

    print("DISCARDED!")

    display(data.discarded[["Frequência", "Entidade", "Log"]])

    print(150 * "=")

    print(data.df.Multiplicidade.sum())

    data.save()

    print(f"Elapsed time: {time.perf_counter() - start} seconds")

DATA


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Cod_Tipo_Antena,Polarização,Ganho_Antena,FC_Antena,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Log
0,173.075,TRACKER DO BRASIL LTDA,50004466071,505805553,Mairiporã,3528502,SP,-23.403056,-46.635833,FB,...,86,V,5.8,<NA>,<NA>,<NA>,<NA>,60.0,NaN,[]
1,173.075,TRACKER DO BRASIL LTDA,50004466071,684490021,Belo Horizonte,3106200,MG,-19.793333,-43.968889,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,NaN,[]
2,173.075,TRACKER DO BRASIL LTDA,50004466071,684587017,Tubarão,4218707,SC,-28.520833,-48.985278,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,25.0,NaN,[]
3,173.075,TRACKER DO BRASIL LTDA,50004466071,684610094,Florianópolis,4205407,SC,-27.588611,-48.533611,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,15.0,NaN,[]
4,173.075,TRACKER DO BRASIL LTDA,50004466071,684649900,João Monlevade,3136207,MG,-19.868889,-43.193333,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,788.000,TELEFONICA BRASIL S.A.,50419058354,1014443412,MACAE,3302403,RJ,-22.623611,-39.989720,FB,...,760,X,14.9,22.0,68.0,0.0,60.0,71.5,0.0,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
12756,1865.000,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,760,X,16.7,25.0,64.0,0.0,206.0,61.47,0.0,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
12757,788.000,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,760,X,14.9,22.0,68.0,0.0,206.0,61.47,0.0,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
12758,1862.500,TELEFONICA BRASIL S.A.,50419058354,1014443471,Vitória,3205309,ES,-21.213889,-39.997222,FB,...,760,X,16.7,25.0,64.0,0.0,187.0,58.05,0.0,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."


DISCARDED!


,Frequência,Entidade,Log
0,459.25000,Cricom do Brasil Ltda,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
1,469.12500,Cricom do Brasil Ltda,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
2,383.12500,LOOPCOM COMERCIO E SERVICOS LTDA - ME,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
3,383.85000,JEVIN COMERCIO E SERVIÇOS LTDA,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
4,459.58750,JEVIN COMERCIO E SERVIÇOS LTDA,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
...,...,...,...
87235,451.86875,AMTRAK COMERCIO E SERVIÇOS DE RADIOCOMUNICAÇÃO...,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
87236,451.86875,AMTRAK COMERCIO E SERVIÇOS DE RADIOCOMUNICAÇÃO...,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
87237,451.86875,AMTRAK COMERCIO E SERVIÇOS DE RADIOCOMUNICAÇÃO...,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."
87238,451.86875,AMTRAK COMERCIO E SERVIÇOS DE RADIOCOMUNICAÇÃO...,"[('Colunas', ['Frequência', 'Entidade', 'Fiste..."


100000
Elapsed time: 24.391028399812058 seconds
